In [10]:
import os
from typing import List
from dotenv import load_dotenv
import openai
import requests
import gradio as gr

In [11]:
# Set your OpenAI API key

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


# Set your weather API key (e.g., OpenWeatherMap)
weather_api_key = "96f2d47a11bf0aa1eb6964cd4cc3a0bc"


OpenAI API Key exists and begins sk-proj-


In [28]:

def get_weather_forecast(location, days):
    # Fetch weather data from OpenWeatherMap API
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        weather_data = response.json()
        forecast = []
        for item in weather_data['list']:
            date = item['dt_txt']
            temp = item['main']['temp']
            description = item['weather'][0]['description']
            forecast.append(f"{date}: {temp}°C, {description}")
        return "\n".join(forecast)
    else:
        return "Sorry, I couldn't fetch the weather data. Please check the location or try again later."

# A generic system message
system_message = "You are a helpful weather forecasting assistant assistant"

def chat_with_llm(location, history):
    # Get weather forecast
    forecast = get_weather_forecast(location, 3)
    
    # Use OpenAI LLM to generate a response
    prompt = f"The user asked for a 3-day weather forecast for {location}. Here is the forecast:\n{forecast}\n\nGenerate a friendly response for the user."
    # messages = [
    #     {"role": "system", "content": system_message},
    #     {"role": "user", "content": prompt}
    #   ]
    
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": prompt}]

    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        max_tokens=600
    )
    
    response = completion.choices[0].message.content
    return response



# Gradio interface
interface = gr.ChatInterface(fn=chat_with_llm, type="messages")
interface.launch()

* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.
